# Notebook 12: Framework Integration

## Best Practices for Agentic AI using MCP

---

**Series:** MCP Server Best Practices - Agentic AI Workflows  
**Notebook:** 12 of 13  
**Level:** Advanced  
**Duration:** ~75 minutes

---

## Learning Objectives

By the end of this notebook, you will:

1. Integrate MCP servers with LangChain
2. Build LangGraph workflows using MCP tools
3. Create CrewAI multi-agent systems with MCP
4. Understand the MCP Adapter pattern
5. Build custom framework integrations
6. Choose the right framework for your use case

## 1. Framework Landscape

### Why Framework Integration?

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    MCP + FRAMEWORKS = POWER                              │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   MCP PROVIDES:                    FRAMEWORKS PROVIDE:                  │
│   ═════════════                    ════════════════════                 │
│   • Standardized tool interface    • Orchestration logic                │
│   • Discovery mechanism            • Memory management                  │
│   • Type-safe schemas              • Prompt templates                   │
│   • Transport abstraction          • Chain composition                  │
│   • Security model                 • Agent patterns                     │
│                                                                          │
│   ─────────────────────────────────────────────────────────────────     │
│                                                                          │
│   TOGETHER:                                                             │
│                                                                          │
│   ┌─────────────────────────────────────────────────────────────────┐   │
│   │                      YOUR APPLICATION                            │   │
│   │                                                                  │   │
│   │   ┌─────────────┐    ┌─────────────┐    ┌─────────────┐        │   │
│   │   │  LangChain  │    │  LangGraph  │    │   CrewAI    │        │   │
│   │   │  Chains     │    │  Workflows  │    │   Agents    │        │   │
│   │   └──────┬──────┘    └──────┬──────┘    └──────┬──────┘        │   │
│   │          │                  │                  │                │   │
│   │          └──────────────────┼──────────────────┘                │   │
│   │                             │                                    │   │
│   │                    ┌────────▼────────┐                          │   │
│   │                    │   MCP ADAPTER   │                          │   │
│   │                    └────────┬────────┘                          │   │
│   │                             │                                    │   │
│   └─────────────────────────────┼────────────────────────────────────┘   │
│                                 │                                       │
│          ┌──────────────────────┼──────────────────────┐                │
│          │                      │                      │                │
│          ▼                      ▼                      ▼                │
│   ┌─────────────┐        ┌─────────────┐        ┌─────────────┐        │
│   │ GitHub MCP  │        │Database MCP │        │ Custom MCP  │        │
│   └─────────────┘        └─────────────┘        └─────────────┘        │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### Framework Comparison

| Feature | LangChain | LangGraph | CrewAI | AutoGen |
|---------|-----------|-----------|--------|----------|
| **Best For** | Chains & RAG | Complex Workflows | Multi-Agent | Conversations |
| **Complexity** | Medium | High | Low | Medium |
| **MCP Support** | Via Adapter | Via Adapter | Via Adapter | Via Adapter |
| **State Mgmt** | Basic | Advanced | Built-in | Built-in |
| **Debugging** | Good | Excellent | Good | Good |

In [ ]:
# Setup for this notebook
import json
from typing import Dict, List, Any, Optional, Callable
from dataclasses import dataclass, field
from datetime import datetime
from abc import ABC, abstractmethod

def pprint(obj: dict, title: str = None):
    if title:
        print(f"\n{'='*60}")
        print(f" {title}")
        print(f"{'='*60}")
    print(json.dumps(obj, indent=2, default=str))

## 2. The MCP Adapter Pattern

### Universal Adapter Design

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    MCP ADAPTER PATTERN                                   │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   Framework Tool Format              MCP Tool Format                    │
│   ═════════════════════              ═══════════════                    │
│                                                                          │
│   LangChain:                         MCP:                               │
│   {                                  {                                  │
│     "name": "search",                 "name": "search",                │
│     "description": "...",             "description": "...",            │
│     "args_schema": Pydantic          "inputSchema": JSON Schema        │
│   }                        ◄────►    }                                  │
│                            ADAPTER                                      │
│   CrewAI:                                                               │
│   @tool decorator                                                       │
│   def search(...):        ◄────►                                        │
│       ...                                                               │
│                                                                          │
│   ─────────────────────────────────────────────────────────────────     │
│                                                                          │
│   ADAPTER RESPONSIBILITIES:                                             │
│                                                                          │
│   1. DISCOVERY: List MCP tools → Framework tool objects                 │
│   2. SCHEMA: JSON Schema → Framework schema (Pydantic, etc.)            │
│   3. INVOCATION: Framework call → MCP tools/call                        │
│   4. RESPONSE: MCP result → Framework expected format                   │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Base MCP Adapter

@dataclass
class MCPTool:
    """Representation of an MCP tool."""
    name: str
    description: str
    input_schema: Dict[str, Any]
    server_name: str


@dataclass
class MCPToolResult:
    """Result from an MCP tool call."""
    tool_name: str
    success: bool
    content: Any
    error: Optional[str] = None


class MCPClient:
    """Simulated MCP client for framework integration demos."""
    
    def __init__(self):
        self.tools: Dict[str, MCPTool] = {}
        self._setup_demo_tools()
    
    def _setup_demo_tools(self):
        """Setup demo tools."""
        self.tools = {
            "search_code": MCPTool(
                name="search_code",
                description="Search for code in repositories",
                input_schema={
                    "type": "object",
                    "properties": {
                        "query": {"type": "string", "description": "Search query"},
                        "language": {"type": "string", "description": "Programming language"}
                    },
                    "required": ["query"]
                },
                server_name="github-mcp"
            ),
            "create_issue": MCPTool(
                name="create_issue",
                description="Create a GitHub issue",
                input_schema={
                    "type": "object",
                    "properties": {
                        "repo": {"type": "string", "description": "Repository name"},
                        "title": {"type": "string", "description": "Issue title"},
                        "body": {"type": "string", "description": "Issue body"}
                    },
                    "required": ["repo", "title"]
                },
                server_name="github-mcp"
            ),
            "query_database": MCPTool(
                name="query_database",
                description="Execute SQL query on database",
                input_schema={
                    "type": "object",
                    "properties": {
                        "sql": {"type": "string", "description": "SQL query"},
                        "database": {"type": "string", "description": "Database name"}
                    },
                    "required": ["sql"]
                },
                server_name="database-mcp"
            )
        }
    
    def list_tools(self) -> List[MCPTool]:
        """List available tools."""
        return list(self.tools.values())
    
    def call_tool(self, name: str, arguments: Dict[str, Any]) -> MCPToolResult:
        """Call a tool."""
        if name not in self.tools:
            return MCPToolResult(
                tool_name=name,
                success=False,
                content=None,
                error=f"Tool not found: {name}"
            )
        
        # Simulate tool execution
        return MCPToolResult(
            tool_name=name,
            success=True,
            content=f"Executed {name} with {arguments}"
        )


class BaseMCPAdapter(ABC):
    """Base adapter for framework integrations."""
    
    def __init__(self, mcp_client: MCPClient):
        self.mcp_client = mcp_client
    
    @abstractmethod
    def convert_tool(self, mcp_tool: MCPTool) -> Any:
        """Convert MCP tool to framework-specific format."""
        pass
    
    @abstractmethod
    def convert_result(self, result: MCPToolResult) -> Any:
        """Convert MCP result to framework-specific format."""
        pass
    
    def get_tools(self) -> List[Any]:
        """Get all tools in framework format."""
        return [self.convert_tool(t) for t in self.mcp_client.list_tools()]


# Demo
mcp_client = MCPClient()
print("MCP Client Tools:")
print("=" * 60)
for tool in mcp_client.list_tools():
    print(f"\n• {tool.name}")
    print(f"  Description: {tool.description}")
    print(f"  Server: {tool.server_name}")

## 3. LangChain Integration

### Converting MCP Tools to LangChain Tools

In [ ]:
# LangChain MCP Adapter

# Note: This is a simplified implementation for demonstration
# In production, you would use the actual langchain package

@dataclass
class LangChainTool:
    """Simplified LangChain tool representation."""
    name: str
    description: str
    args_schema: Dict[str, Any]  # Would be Pydantic in real LangChain
    func: Callable
    
    def invoke(self, input_data: Dict[str, Any]) -> str:
        """Invoke the tool."""
        return self.func(input_data)


class LangChainMCPAdapter(BaseMCPAdapter):
    """Adapter to use MCP tools with LangChain."""
    
    def convert_tool(self, mcp_tool: MCPTool) -> LangChainTool:
        """Convert MCP tool to LangChain tool."""
        
        # Create a wrapper function that calls MCP
        def tool_func(input_data: Dict[str, Any]) -> str:
            result = self.mcp_client.call_tool(mcp_tool.name, input_data)
            if result.success:
                return str(result.content)
            else:
                return f"Error: {result.error}"
        
        return LangChainTool(
            name=mcp_tool.name,
            description=mcp_tool.description,
            args_schema=mcp_tool.input_schema,
            func=tool_func
        )
    
    def convert_result(self, result: MCPToolResult) -> str:
        """Convert MCP result to LangChain format."""
        if result.success:
            return str(result.content)
        return f"Error: {result.error}"


# Production LangChain integration code
LANGCHAIN_MCP_CODE = '''
# pip install langchain langchain-anthropic mcp

from langchain_core.tools import StructuredTool
from langchain_anthropic import ChatAnthropic
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import asyncio

class LangChainMCPIntegration:
    """Production LangChain + MCP integration."""
    
    def __init__(self, server_command: str, server_args: list):
        self.server_params = StdioServerParameters(
            command=server_command,
            args=server_args
        )
        self.tools = []
    
    async def connect_and_load_tools(self):
        """Connect to MCP server and load tools."""
        async with stdio_client(self.server_params) as (read, write):
            async with ClientSession(read, write) as session:
                await session.initialize()
                
                # Get tools from MCP server
                tools_response = await session.list_tools()
                
                for mcp_tool in tools_response.tools:
                    # Convert to LangChain StructuredTool
                    lc_tool = self._convert_to_langchain(mcp_tool, session)
                    self.tools.append(lc_tool)
        
        return self.tools
    
    def _convert_to_langchain(self, mcp_tool, session) -> StructuredTool:
        """Convert MCP tool to LangChain StructuredTool."""
        
        async def call_mcp_tool(**kwargs):
            result = await session.call_tool(mcp_tool.name, kwargs)
            return result.content[0].text if result.content else ""
        
        # Create sync wrapper for LangChain
        def sync_wrapper(**kwargs):
            return asyncio.run(call_mcp_tool(**kwargs))
        
        return StructuredTool.from_function(
            func=sync_wrapper,
            name=mcp_tool.name,
            description=mcp_tool.description,
            args_schema=self._json_schema_to_pydantic(mcp_tool.inputSchema)
        )
    
    def create_agent(self, model_name: str = "claude-sonnet-4-20250514"):
        """Create a LangChain agent with MCP tools."""
        llm = ChatAnthropic(model=model_name)
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful assistant with access to tools."),
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}")
        ])
        
        agent = create_tool_calling_agent(llm, self.tools, prompt)
        return AgentExecutor(agent=agent, tools=self.tools, verbose=True)

# Usage
integration = LangChainMCPIntegration("python", ["github_mcp_server.py"])
await integration.connect_and_load_tools()
agent = integration.create_agent()
result = agent.invoke({"input": "Search for Python authentication examples"})
'''

# Demo
adapter = LangChainMCPAdapter(mcp_client)
langchain_tools = adapter.get_tools()

print("LangChain MCP Adapter Demo:")
print("=" * 60)
print(f"\nConverted {len(langchain_tools)} MCP tools to LangChain format:")

for tool in langchain_tools:
    print(f"\n• {tool.name}")
    print(f"  Description: {tool.description}")
    
    # Test invocation
    result = tool.invoke({"query": "test query"})
    print(f"  Test result: {result}")

print("\n" + "=" * 60)
print("Production LangChain Integration Code:")
print("=" * 60)
print(LANGCHAIN_MCP_CODE)

## 4. LangGraph Integration

### Building Stateful Workflows with MCP

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    LANGGRAPH + MCP WORKFLOW                              │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ┌──────────────────────────────────────────────────────────────┐      │
│   │                    LANGGRAPH STATE MACHINE                    │      │
│   │                                                               │      │
│   │   ┌─────────┐     ┌─────────┐     ┌─────────┐               │      │
│   │   │  Start  │────►│ Research│────►│ Analyze │               │      │
│   │   └─────────┘     └────┬────┘     └────┬────┘               │      │
│   │                        │               │                     │      │
│   │                        │  MCP Tools    │  MCP Tools          │      │
│   │                        ▼               ▼                     │      │
│   │                   [search_code]   [query_db]                │      │
│   │                   [fetch_docs]    [analyze]                 │      │
│   │                        │               │                     │      │
│   │                        └───────┬───────┘                     │      │
│   │                                │                             │      │
│   │                                ▼                             │      │
│   │                         ┌───────────┐                        │      │
│   │                         │  Decide   │                        │      │
│   │                         └─────┬─────┘                        │      │
│   │                    ┌──────────┼──────────┐                   │      │
│   │                    ▼          ▼          ▼                   │      │
│   │               [Create]    [Update]    [End]                 │      │
│   │               [create_issue]  [update_doc]                  │      │
│   │                                                               │      │
│   └───────────────────────────────────────────────────────────────┘      │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# LangGraph MCP Integration

# Simplified LangGraph-style state machine for demo
from enum import Enum
from typing import TypedDict, Annotated

class WorkflowState(TypedDict):
    """State for the workflow."""
    messages: List[str]
    current_step: str
    research_results: Optional[Dict]
    analysis_results: Optional[Dict]
    final_action: Optional[str]


class MCPWorkflowNode:
    """A node in the workflow that uses MCP tools."""
    
    def __init__(self, name: str, mcp_client: MCPClient, tools: List[str]):
        self.name = name
        self.mcp_client = mcp_client
        self.tools = tools
    
    def __call__(self, state: WorkflowState) -> WorkflowState:
        """Execute the node."""
        state["current_step"] = self.name
        state["messages"].append(f"Executing {self.name}")
        
        # Execute tools
        for tool_name in self.tools:
            result = self.mcp_client.call_tool(tool_name, {"query": "example"})
            state["messages"].append(f"  Tool {tool_name}: {result.success}")
        
        return state


class MCPWorkflow:
    """Simplified LangGraph-style workflow with MCP."""
    
    def __init__(self, mcp_client: MCPClient):
        self.mcp_client = mcp_client
        self.nodes: Dict[str, MCPWorkflowNode] = {}
        self.edges: Dict[str, str] = {}
        self.conditional_edges: Dict[str, Callable] = {}
    
    def add_node(self, name: str, tools: List[str]):
        """Add a node to the workflow."""
        self.nodes[name] = MCPWorkflowNode(name, self.mcp_client, tools)
    
    def add_edge(self, from_node: str, to_node: str):
        """Add a direct edge."""
        self.edges[from_node] = to_node
    
    def add_conditional_edge(self, from_node: str, condition: Callable):
        """Add a conditional edge."""
        self.conditional_edges[from_node] = condition
    
    def run(self, initial_state: WorkflowState) -> WorkflowState:
        """Run the workflow."""
        state = initial_state
        current = "start"
        
        while current != "end":
            if current in self.nodes:
                state = self.nodes[current](state)
            
            # Determine next node
            if current in self.conditional_edges:
                current = self.conditional_edges[current](state)
            elif current in self.edges:
                current = self.edges[current]
            else:
                current = "end"
        
        return state


# Production LangGraph code
LANGGRAPH_MCP_CODE = '''
# pip install langgraph langchain-anthropic mcp

from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_anthropic import ChatAnthropic
from typing import TypedDict, Annotated
import operator

class AgentState(TypedDict):
    messages: Annotated[list, operator.add]
    research_complete: bool
    action_taken: bool

class MCPLangGraphAgent:
    """LangGraph agent with MCP tool integration."""
    
    def __init__(self, mcp_tools: list):
        self.tools = mcp_tools
        self.llm = ChatAnthropic(model="claude-sonnet-4-20250514").bind_tools(mcp_tools)
        self.tool_node = ToolNode(mcp_tools)
    
    def build_graph(self) -> StateGraph:
        """Build the LangGraph workflow."""
        workflow = StateGraph(AgentState)
        
        # Add nodes
        workflow.add_node("research", self.research_node)
        workflow.add_node("tools", self.tool_node)
        workflow.add_node("analyze", self.analyze_node)
        workflow.add_node("act", self.action_node)
        
        # Add edges
        workflow.set_entry_point("research")
        workflow.add_edge("research", "tools")
        workflow.add_conditional_edges(
            "tools",
            self.should_continue,
            {
                "continue": "analyze",
                "end": END
            }
        )
        workflow.add_edge("analyze", "act")
        workflow.add_edge("act", END)
        
        return workflow.compile()
    
    def research_node(self, state: AgentState):
        """Research phase using MCP tools."""
        response = self.llm.invoke(state["messages"])
        return {"messages": [response]}
    
    def analyze_node(self, state: AgentState):
        """Analyze research results."""
        # Analyze and decide on action
        return {"research_complete": True}
    
    def action_node(self, state: AgentState):
        """Take action based on analysis."""
        response = self.llm.invoke(state["messages"])
        return {"messages": [response], "action_taken": True}
    
    def should_continue(self, state: AgentState) -> str:
        """Decide whether to continue or end."""
        last_message = state["messages"][-1]
        if hasattr(last_message, "tool_calls") and last_message.tool_calls:
            return "continue"
        return "end"

# Usage
agent = MCPLangGraphAgent(mcp_tools)
graph = agent.build_graph()
result = graph.invoke({"messages": ["Find and fix the authentication bug"]})
'''

# Demo
workflow = MCPWorkflow(mcp_client)

# Build workflow
workflow.add_node("research", ["search_code"])
workflow.add_node("analyze", ["query_database"])
workflow.add_node("act", ["create_issue"])

workflow.add_edge("start", "research")
workflow.add_edge("research", "analyze")
workflow.add_edge("analyze", "act")
workflow.add_edge("act", "end")

# Run workflow
initial_state: WorkflowState = {
    "messages": [],
    "current_step": "start",
    "research_results": None,
    "analysis_results": None,
    "final_action": None
}

final_state = workflow.run(initial_state)

print("LangGraph MCP Workflow Demo:")
print("=" * 60)
print("\nWorkflow execution trace:")
for msg in final_state["messages"]:
    print(f"  {msg}")

## 5. CrewAI Integration

### Multi-Agent Systems with MCP

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    CREWAI + MCP MULTI-AGENT SYSTEM                       │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ┌───────────────────────────────────────────────────────────────┐     │
│   │                         CREW                                   │     │
│   │                                                                │     │
│   │   ┌─────────────┐  ┌─────────────┐  ┌─────────────┐          │     │
│   │   │  RESEARCHER │  │  DEVELOPER  │  │  REVIEWER   │          │     │
│   │   │    Agent    │  │    Agent    │  │    Agent    │          │     │
│   │   │             │  │             │  │             │          │     │
│   │   │ MCP Tools:  │  │ MCP Tools:  │  │ MCP Tools:  │          │     │
│   │   │ • search    │  │ • github    │  │ • linter    │          │     │
│   │   │ • fetch_doc │  │ • code_exec │  │ • test      │          │     │
│   │   └──────┬──────┘  └──────┬──────┘  └──────┬──────┘          │     │
│   │          │                │                │                  │     │
│   │          └────────────────┼────────────────┘                  │     │
│   │                           │                                   │     │
│   │                    ┌──────▼──────┐                            │     │
│   │                    │  TASK QUEUE │                            │     │
│   │                    │             │                            │     │
│   │                    │ 1. Research │                            │     │
│   │                    │ 2. Implement│                            │     │
│   │                    │ 3. Review   │                            │     │
│   │                    └─────────────┘                            │     │
│   │                                                                │     │
│   └────────────────────────────────────────────────────────────────┘     │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# CrewAI MCP Integration

@dataclass
class CrewAgent:
    """Simplified CrewAI agent for demo."""
    name: str
    role: str
    goal: str
    tools: List[str]  # MCP tool names
    
    def execute_task(self, task: str, mcp_client: MCPClient) -> str:
        """Execute a task using available tools."""
        results = []
        for tool_name in self.tools:
            result = mcp_client.call_tool(tool_name, {"task": task})
            results.append(f"{tool_name}: {result.content}")
        return f"{self.name} completed: {'; '.join(results)}"


@dataclass
class CrewTask:
    """A task for the crew."""
    description: str
    assigned_agent: str
    expected_output: str


class MCPCrew:
    """Simplified CrewAI with MCP integration."""
    
    def __init__(self, mcp_client: MCPClient):
        self.mcp_client = mcp_client
        self.agents: Dict[str, CrewAgent] = {}
        self.tasks: List[CrewTask] = []
    
    def add_agent(self, agent: CrewAgent):
        """Add an agent to the crew."""
        self.agents[agent.name] = agent
    
    def add_task(self, task: CrewTask):
        """Add a task to the queue."""
        self.tasks.append(task)
    
    def kickoff(self) -> List[str]:
        """Execute all tasks."""
        results = []
        for task in self.tasks:
            agent = self.agents.get(task.assigned_agent)
            if agent:
                result = agent.execute_task(task.description, self.mcp_client)
                results.append(result)
            else:
                results.append(f"No agent found for {task.assigned_agent}")
        return results


# Production CrewAI code
CREWAI_MCP_CODE = '''
# pip install crewai crewai-tools mcp

from crewai import Agent, Task, Crew, Process
from crewai_tools import tool
from mcp import ClientSession

class MCPToolWrapper:
    """Wrapper to convert MCP tools to CrewAI tools."""
    
    def __init__(self, mcp_session: ClientSession):
        self.session = mcp_session
    
    def create_crewai_tool(self, mcp_tool):
        """Create a CrewAI tool from an MCP tool."""
        
        @tool(mcp_tool.name)
        def mcp_tool_wrapper(**kwargs) -> str:
            """MCP tool wrapper."""
            result = asyncio.run(
                self.session.call_tool(mcp_tool.name, kwargs)
            )
            return result.content[0].text if result.content else ""
        
        mcp_tool_wrapper.__doc__ = mcp_tool.description
        return mcp_tool_wrapper

# Create agents with MCP tools
researcher = Agent(
    role="Senior Researcher",
    goal="Find relevant code and documentation",
    backstory="Expert at finding information in codebases",
    tools=[search_code_tool, fetch_docs_tool],  # MCP tools
    verbose=True
)

developer = Agent(
    role="Senior Developer",
    goal="Implement solutions based on research",
    backstory="Expert Python developer",
    tools=[github_tool, code_executor_tool],  # MCP tools
    verbose=True
)

reviewer = Agent(
    role="Code Reviewer",
    goal="Ensure code quality and correctness",
    backstory="Meticulous code reviewer",
    tools=[linter_tool, test_runner_tool],  # MCP tools
    verbose=True
)

# Define tasks
research_task = Task(
    description="Research authentication best practices",
    agent=researcher,
    expected_output="Summary of best practices with code examples"
)

implement_task = Task(
    description="Implement authentication module",
    agent=developer,
    expected_output="Working authentication code",
    context=[research_task]
)

review_task = Task(
    description="Review and test the implementation",
    agent=reviewer,
    expected_output="Code review with test results",
    context=[implement_task]
)

# Create and run crew
crew = Crew(
    agents=[researcher, developer, reviewer],
    tasks=[research_task, implement_task, review_task],
    process=Process.sequential,
    verbose=True
)

result = crew.kickoff()
'''

# Demo
crew = MCPCrew(mcp_client)

# Add agents
crew.add_agent(CrewAgent(
    name="researcher",
    role="Senior Researcher",
    goal="Find relevant information",
    tools=["search_code"]
))

crew.add_agent(CrewAgent(
    name="developer",
    role="Senior Developer",
    goal="Implement solutions",
    tools=["create_issue", "query_database"]
))

# Add tasks
crew.add_task(CrewTask(
    description="Research authentication patterns",
    assigned_agent="researcher",
    expected_output="Research summary"
))

crew.add_task(CrewTask(
    description="Create implementation issue",
    assigned_agent="developer",
    expected_output="GitHub issue created"
))

# Execute
results = crew.kickoff()

print("CrewAI MCP Integration Demo:")
print("=" * 60)
print("\nCrew execution results:")
for result in results:
    print(f"  • {result}")

## 6. Framework Selection Guide

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    FRAMEWORK SELECTION DECISION TREE                     │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   Start: What's your primary use case?                                  │
│                    │                                                    │
│       ┌────────────┼────────────┬─────────────┐                         │
│       │            │            │             │                         │
│       ▼            ▼            ▼             ▼                         │
│   Simple        Complex      Multi-       Chat-based                    │
│   Chains        Workflows    Agent        Assistants                    │
│       │            │            │             │                         │
│       ▼            ▼            ▼             ▼                         │
│   ┌────────┐  ┌────────┐  ┌────────┐  ┌────────┐                       │
│   │LANGCHAIN│  │LANGGRAPH│  │ CREWAI │  │ AUTOGEN│                       │
│   └────────┘  └────────┘  └────────┘  └────────┘                       │
│                                                                          │
│   ─────────────────────────────────────────────────────────────────     │
│                                                                          │
│   DETAILED COMPARISON:                                                  │
│                                                                          │
│   LangChain                        LangGraph                            │
│   ═════════                        ═════════                            │
│   ✅ Great for RAG                 ✅ Complex state machines            │
│   ✅ Easy tool integration         ✅ Conditional branching             │
│   ✅ Large ecosystem               ✅ Cyclic workflows                  │
│   ❌ Limited control flow          ✅ Built-in persistence              │
│   ❌ Debugging can be hard         ❌ Steeper learning curve            │
│                                                                          │
│   CrewAI                           AutoGen                              │
│   ══════                           ═══════                              │
│   ✅ Role-based agents             ✅ Conversational agents             │
│   ✅ Easy to understand            ✅ Human-in-the-loop                 │
│   ✅ Built-in collaboration        ✅ Code execution                    │
│   ❌ Less flexible                 ❌ Less control over flow            │
│   ❌ Newer ecosystem               ❌ Microsoft-centric                 │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

## 7. Key Takeaways

### Integration Patterns Summary

| Framework | MCP Integration | Best Use Case |
|-----------|-----------------|---------------|
| **LangChain** | StructuredTool wrapper | RAG, simple chains |
| **LangGraph** | ToolNode in graph | Complex state machines |
| **CrewAI** | @tool decorator | Multi-agent collaboration |
| **AutoGen** | Function tools | Conversational agents |

### Best Practices

1. **Use the Adapter Pattern** - Don't couple directly to MCP internals
2. **Handle Async Properly** - MCP is async; frameworks may need sync wrappers
3. **Error Handling** - Wrap MCP errors in framework-appropriate exceptions
4. **Tool Discovery** - Refresh tools periodically in long-running applications
5. **Testing** - Test MCP tools independently before framework integration

### Next Notebook

In **Notebook 13: Production Deployment**, we'll cover:
- Container deployment strategies
- Kubernetes orchestration
- CI/CD pipelines for MCP
- Monitoring and alerting

## 8. Exercises

### Exercise 1: Build a Custom Adapter

Create an adapter for a hypothetical framework:

In [ ]:
# Exercise 1: Create a custom adapter

class MyFrameworkAdapter(BaseMCPAdapter):
    """Adapter for MyFramework.
    
    MyFramework expects tools in this format:
    {
        "id": "unique_id",
        "label": "Human readable name",
        "params": [{"name": "...", "type": "..."}],
        "handler": callable
    }
    """
    
    def convert_tool(self, mcp_tool: MCPTool) -> Dict:
        """Convert MCP tool to MyFramework format."""
        # Your implementation here
        pass
    
    def convert_result(self, result: MCPToolResult) -> Dict:
        """Convert MCP result to MyFramework format."""
        # Your implementation here
        pass

# Test your adapter
# adapter = MyFrameworkAdapter(mcp_client)
# tools = adapter.get_tools()
# print(tools)

### Exercise 2: Build a LangGraph Workflow

Create a code review workflow with these steps:
1. Fetch code from repository
2. Run linter
3. Run tests
4. Generate review summary

In [ ]:
# Exercise 2: Build a code review workflow

class CodeReviewWorkflow:
    """LangGraph-style code review workflow."""
    
    def __init__(self, mcp_client: MCPClient):
        self.mcp_client = mcp_client
        # Your implementation here
    
    def build_graph(self):
        """Build the workflow graph."""
        # Your implementation here
        pass
    
    def run(self, repo: str, pr_number: int):
        """Run the code review."""
        # Your implementation here
        pass

# Test your workflow
# workflow = CodeReviewWorkflow(mcp_client)
# result = workflow.run("owner/repo", 123)

---

## References

- [LangChain Tools Documentation](https://python.langchain.com/docs/modules/tools/)
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [CrewAI Documentation](https://docs.crewai.com/)
- [AutoGen Documentation](https://microsoft.github.io/autogen/)
- [MCP SDK Python](https://github.com/modelcontextprotocol/python-sdk)

---

**Next Notebook:** [13_Production_Deployment.ipynb](./13_Production_Deployment.ipynb)